In [1]:
import pandas as pd
import numpy as np
import yaml

In [2]:
regs=pd.read_csv('regs.1.6.0.csv')
maps=pd.read_csv('maps.1.6.0.csv')

maps['BlockMap Instance Name']=np.array([v[:-4] for v in maps['BlockMap Instance Name'].values])

repeatedGroups=[]
for x in regs['Register Name']:
    if ('_xx_ALL' in x) and not (x in repeatedGroups):
        repeatedGroups.append(x)
        
maps.set_index('BlockMap Instance Name',inplace=True)
regs.set_index('Register Name',inplace=True)

a=regs.loc[repeatedGroups].reset_index()

values=[]
for i in range(12):
    values.append(a.copy())
    values[-1]['Register Name'] = a['Register Name'].str.replace('xx_ALL',f'{i:02n}_ALL')

In [3]:
fullRegs=pd.concat(values+[regs.drop(repeatedGroups).reset_index()])
fullRegs.set_index('Register Name',inplace=True)
fullRegs['Register Address']=maps['BlockMap Instance Address']
fullRegs.sort_values('Register Address',inplace=True)

blockName=fullRegs.index.values

blockName=[x.split('_w')[0] for x in blockName]

fullRegs['BlockName']=blockName
first=(fullRegs.BlockName!=fullRegs.BlockName.shift(1))
fullRegs.loc[first,'addr_base']=fullRegs.loc[first,'Register Address']
fullRegs.addr_base=fullRegs.addr_base.fillna(method='ffill')

fullRegs['addr_offset']=(fullRegs['Register Address'].apply(int,base=16)-fullRegs['addr_base'].apply(int,base=16)).apply(lambda x: f'0x{x:02x}')

In [4]:
blocks=fullRegs.BlockName.unique()

registerMap={'ninput': 12,
             'noutput': 6,
             'WO':{},
             'RW':{},
             'RO':{},
            }

newRegList=[]

for b in blocks:
    df=fullRegs.loc[fullRegs.BlockName==b].sort_values(['Register Address','Field Offset'])
    block=b[3:]
    addr_base=df.addr_base[0]
    reg_access=df['Register Access'][0]

    #find fields that span multiple bytes, and get sum of field widths
    multiByteRegs=df['Field Name'].value_counts()>1
    multiByteRegs=multiByteRegs.index[multiByteRegs.values]
    multiRegByteWidths=df.groupby('Field Name').sum()['Field Width'][multiByteRegs]

    df.reset_index(inplace=True)
    #if registers are split over multiple bytes, assigning it to the lowest word
    for r in multiByteRegs:
        x=df[df['Field Name']==r]['Register Name'].values
        df.loc[df['Register Name'].isin(x),'Register Name']=df.loc[df['Register Name'].isin(x)]['Register Name'].values[0]


    registers={}

    u=df['Register Name'].unique()

    for b in u[:]:
        print(b)
        d=df.loc[df['Register Name']==b]
        widths=d.groupby('Field Name').sum()['Field Width']

        d=pd.merge(d,widths,left_on='Field Name',right_index=True)
        d.rename(columns={'Field Width_x':'Field Width'},inplace=True)
        d['Field Width']=d['Field Width_y']
        d.drop('Field Width_y',axis=1,inplace=True)

        isDuplicate=d['Field Name']==d['Field Name'].shift()
        d=d.drop(d[isDuplicate].index)
        d['Field Offset']=d['Field Width'].cumsum()-d['Field Width']

        param=d['Field Name'].values
        dropRSVD=~np.char.startswith(param.astype(np.string_),b'RSVD')

        param=param[dropRSVD]
        param_shift=d['Field Offset'].values[dropRSVD]
        param_mask=d['Field Width'].apply(lambda x: f'0x{(2**x)-1:x}').values[dropRSVD]
        param_resetVal=d['Field Reset Value'].apply(int,base=16).values[dropRSVD]

        default_value=f'{(param_resetVal<<param_shift).sum()}'
        size_byte=int(d['Field Width'].sum()/8)
        d['size_byte']=size_byte
        if len(param)==1:
            registers[param[0]]={'addr_offset':d['addr_offset'].values[0],
                                 'size_byte':size_byte,
                                 'value':default_value}
        else:
            byteNum=f"word_{b.split('_w')[1]}"
            registers[byteNum]={'addr_offset':d['addr_offset'].values[0],                
                                'size_byte':size_byte,
                                'value':default_value,
                                'params':{param[i]:{'param_shift':f'{param_shift[i]}','param_mask':param_mask[i]} for i in range(len(param))}
                               }
        newRegList.append(d)

    registerMap[reg_access][block]={'addr_base':addr_base,
                                    'registers':registers,
                                   }


/var/folders/67/xw3dpsz136g3dxp6m1g312hh0000gq/T/ipykernel_30505/3473895911.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  multiRegByteWidths=df.groupby('Field Name').sum()['Field Width'][multiByteRegs]
/var/folders/67/xw3dpsz136g3dxp6m1g312hh0000gq/T/ipykernel_30505/3473895911.py:37: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  widths=d.groupby('Field Name').sum()['Field Width']
/var/folders/67/xw3dpsz136g3dxp6m1g312hh0000gq/T/ipykernel_30505/3473895911.py:37: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False

RW_CHAL_00_ALL_w0
RW_CHAL_00_ALL_w1
RW_CHAL_00_ALL_w2
RW_CHAL_00_ALL_w4
RW_CHAL_00_ALL_w8
RW_CHAL_00_ALL_w12
RW_CHAL_00_ALL_w16
RO_CHAL_00_ALL_w0
RO_CHAL_00_ALL_w1
RO_CHAL_00_ALL_w2
RO_CHAL_00_ALL_w26
RO_CHAL_00_ALL_w28
RO_CHAL_00_ALL_w29
RO_CHAL_00_ALL_w30
RO_CHAL_00_ALL_w31
RO_CHAL_00_ALL_w32
RO_CHAL_00_ALL_w33
RO_CHAL_00_ALL_w34
RW_CHAL_01_ALL_w0
RW_CHAL_01_ALL_w1
RW_CHAL_01_ALL_w2
RW_CHAL_01_ALL_w4
RW_CHAL_01_ALL_w8
RW_CHAL_01_ALL_w12
RW_CHAL_01_ALL_w16
RO_CHAL_01_ALL_w0
RO_CHAL_01_ALL_w1
RO_CHAL_01_ALL_w2
RO_CHAL_01_ALL_w26
RO_CHAL_01_ALL_w28
RO_CHAL_01_ALL_w29
RO_CHAL_01_ALL_w30
RO_CHAL_01_ALL_w31
RO_CHAL_01_ALL_w32
RO_CHAL_01_ALL_w33
RO_CHAL_01_ALL_w34
RW_CHAL_02_ALL_w0
RW_CHAL_02_ALL_w1
RW_CHAL_02_ALL_w2
RW_CHAL_02_ALL_w4
RW_CHAL_02_ALL_w8
RW_CHAL_02_ALL_w12
RW_CHAL_02_ALL_w16
RO_CHAL_02_ALL_w0
RO_CHAL_02_ALL_w1
RO_CHAL_02_ALL_w2
RO_CHAL_02_ALL_w26
RO_CHAL_02_ALL_w28
RO_CHAL_02_ALL_w29
RO_CHAL_02_ALL_w30
RO_CHAL_02_ALL_w31
RO_CHAL_02_ALL_w32
RO_CHAL_02_ALL_w33
RO_CHAL_02_ALL_w3

/var/folders/67/xw3dpsz136g3dxp6m1g312hh0000gq/T/ipykernel_30505/3473895911.py:37: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  widths=d.groupby('Field Name').sum()['Field Width']
/var/folders/67/xw3dpsz136g3dxp6m1g312hh0000gq/T/ipykernel_30505/3473895911.py:37: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  widths=d.groupby('Field Name').sum()['Field Width']
/var/folders/67/xw3dpsz136g3dxp6m1g312hh0000gq/T/ipykernel_30505/3473895911.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_onl

RW_CHAL_05_ALL_w0
RW_CHAL_05_ALL_w1
RW_CHAL_05_ALL_w2
RW_CHAL_05_ALL_w4
RW_CHAL_05_ALL_w8
RW_CHAL_05_ALL_w12
RW_CHAL_05_ALL_w16
RO_CHAL_05_ALL_w0
RO_CHAL_05_ALL_w1
RO_CHAL_05_ALL_w2
RO_CHAL_05_ALL_w26
RO_CHAL_05_ALL_w28
RO_CHAL_05_ALL_w29
RO_CHAL_05_ALL_w30
RO_CHAL_05_ALL_w31
RO_CHAL_05_ALL_w32
RO_CHAL_05_ALL_w33
RO_CHAL_05_ALL_w34
RW_CHAL_06_ALL_w0
RW_CHAL_06_ALL_w1
RW_CHAL_06_ALL_w2
RW_CHAL_06_ALL_w4
RW_CHAL_06_ALL_w8
RW_CHAL_06_ALL_w12
RW_CHAL_06_ALL_w16
RO_CHAL_06_ALL_w0
RO_CHAL_06_ALL_w1
RO_CHAL_06_ALL_w2
RO_CHAL_06_ALL_w26
RO_CHAL_06_ALL_w28
RO_CHAL_06_ALL_w29
RO_CHAL_06_ALL_w30
RO_CHAL_06_ALL_w31
RO_CHAL_06_ALL_w32
RO_CHAL_06_ALL_w33
RO_CHAL_06_ALL_w34
RW_CHAL_07_ALL_w0
RW_CHAL_07_ALL_w1
RW_CHAL_07_ALL_w2
RW_CHAL_07_ALL_w4
RW_CHAL_07_ALL_w8
RW_CHAL_07_ALL_w12
RW_CHAL_07_ALL_w16
RO_CHAL_07_ALL_w0
RO_CHAL_07_ALL_w1
RO_CHAL_07_ALL_w2
RO_CHAL_07_ALL_w26
RO_CHAL_07_ALL_w28
RO_CHAL_07_ALL_w29
RO_CHAL_07_ALL_w30
RO_CHAL_07_ALL_w31
RO_CHAL_07_ALL_w32
RO_CHAL_07_ALL_w33
RO_CHAL_07_ALL_w3

/var/folders/67/xw3dpsz136g3dxp6m1g312hh0000gq/T/ipykernel_30505/3473895911.py:37: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  widths=d.groupby('Field Name').sum()['Field Width']
/var/folders/67/xw3dpsz136g3dxp6m1g312hh0000gq/T/ipykernel_30505/3473895911.py:37: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  widths=d.groupby('Field Name').sum()['Field Width']
/var/folders/67/xw3dpsz136g3dxp6m1g312hh0000gq/T/ipykernel_30505/3473895911.py:37: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_onl

...

/var/folders/67/xw3dpsz136g3dxp6m1g312hh0000gq/T/ipykernel_30505/3473895911.py:21: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  multiRegByteWidths=df.groupby('Field Name').sum()['Field Width'][multiByteRegs]
/var/folders/67/xw3dpsz136g3dxp6m1g312hh0000gq/T/ipykernel_30505/3473895911.py:37: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  widths=d.groupby('Field Name').sum()['Field Width']
/var/folders/67/xw3dpsz136g3dxp6m1g312hh0000gq/T/ipykernel_30505/3473895911.py:37: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False

RW_ZERO_SUPPRESS_M1_11_ALL_w64
RW_ZERO_SUPPRESS_M1_11_ALL_w66
RW_ZERO_SUPPRESS_M1_11_ALL_w68
RW_ZERO_SUPPRESS_M1_11_ALL_w70
RW_ZERO_SUPPRESS_M1_11_ALL_w72
RO_ZERO_SUPPRESS_M1_ALL_w0
WO_WATCHDOG_ALL_w0
RW_WATCHDOG_ALL_w0
RW_WATCHDOG_ALL_w8
RW_WATCHDOG_ALL_w16
RW_WATCHDOG_ALL_w17
RW_WATCHDOG_ALL_w18
RW_WATCHDOG_ALL_w19
RW_WATCHDOG_ALL_w20
RW_WATCHDOG_ALL_w21
RW_WATCHDOG_ALL_w22
RW_WATCHDOG_ALL_w23
RW_WATCHDOG_ALL_w24
RW_WATCHDOG_ALL_w25
RW_WATCHDOG_ALL_w26
RW_WATCHDOG_ALL_w27
RW_WATCHDOG_ALL_w28
RW_WATCHDOG_ALL_w29
RW_WATCHDOG_ALL_w30
RW_WATCHDOG_ALL_w31
RW_WATCHDOG_ALL_w32
RW_WATCHDOG_ALL_w33
RW_WATCHDOG_ALL_w34
RW_WATCHDOG_ALL_w35
RW_WATCHDOG_ALL_w36
RW_WATCHDOG_ALL_w37
RW_WATCHDOG_ALL_w38
RW_WATCHDOG_ALL_w39
RW_WATCHDOG_ALL_w40
RW_WATCHDOG_ALL_w41
RW_WATCHDOG_ALL_w43
RW_WATCHDOG_ALL_w44
RW_WATCHDOG_ALL_w45
RW_WATCHDOG_ALL_w46
RW_WATCHDOG_ALL_w47
RW_WATCHDOG_ALL_w48
RW_WATCHDOG_ALL_w49
RW_WATCHDOG_ALL_w50
RW_WATCHDOG_ALL_w51
RW_WATCHDOG_ALL_w52
RW_WATCHDOG_ALL_w53
RW_WATCHDOG_ALL_w54
R

/var/folders/67/xw3dpsz136g3dxp6m1g312hh0000gq/T/ipykernel_30505/3473895911.py:37: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  widths=d.groupby('Field Name').sum()['Field Width']
/var/folders/67/xw3dpsz136g3dxp6m1g312hh0000gq/T/ipykernel_30505/3473895911.py:37: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  widths=d.groupby('Field Name').sum()['Field Width']
/var/folders/67/xw3dpsz136g3dxp6m1g312hh0000gq/T/ipykernel_30505/3473895911.py:37: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_onl

RO_WATCHDOG_SP_w6
RO_WATCHDOG_SP_w8
RO_WATCHDOG_HAM_w0
RO_WATCHDOG_HAM_w2
RO_WATCHDOG_HAM_w4
RO_WATCHDOG_HAM_w6
RO_WATCHDOG_HAM_w8
RO_WATCHDOG_HAM_w10
RO_WATCHDOG_HAM_w12
RO_WATCHDOG_HAM_w14
RO_WATCHDOG_HAM_w16
RO_WATCHDOG_HAM_w18
RO_WATCHDOG_HAM_w20
RO_WATCHDOG_HAM_w22
RO_WATCHDOG_HAM_w24
RO_WATCHDOG_HAM_w28
RO_WATCHDOG_HAM_w32
RO_WATCHDOG_HAM_w36
RO_WATCHDOG_HAM_w40
RO_WATCHDOG_HAM_w44
RO_WATCHDOG_HAM_w48
RO_WATCHDOG_HAM_w52
RO_WATCHDOG_HAM_w56
RO_WATCHDOG_HAM_w60
RO_WATCHDOG_HAM_w64
RO_WATCHDOG_HAM_w68
RO_WATCHDOG_SUBP_CRC_w0
RO_WATCHDOG_SUBP_CRC_w2
RO_WATCHDOG_SUBP_CRC_w4
RO_WATCHDOG_SUBP_CRC_w6
RO_WATCHDOG_SUBP_CRC_w8
RO_WATCHDOG_SUBP_CRC_w10
RO_WATCHDOG_SUBP_CRC_w12
RO_WATCHDOG_SUBP_CRC_w14
RO_WATCHDOG_SUBP_CRC_w16
RO_WATCHDOG_SUBP_CRC_w18
RO_WATCHDOG_SUBP_CRC_w20
RO_WATCHDOG_SUBP_CRC_w22
RO_WATCHDOG_SUBP_CRC_w24
RO_WATCHDOG_SUBP_CRC_w28
RO_WATCHDOG_SUBP_CRC_w32
RO_WATCHDOG_SUBP_CRC_w36
RO_WATCHDOG_SUBP_CRC_w40
RO_WATCHDOG_SUBP_CRC_w44
RO_WATCHDOG_SUBP_CRC_w48
RO_WATCHDOG_SUBP_CR

/var/folders/67/xw3dpsz136g3dxp6m1g312hh0000gq/T/ipykernel_30505/3473895911.py:37: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  widths=d.groupby('Field Name').sum()['Field Width']
/var/folders/67/xw3dpsz136g3dxp6m1g312hh0000gq/T/ipykernel_30505/3473895911.py:37: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  widths=d.groupby('Field Name').sum()['Field Width']
/var/folders/67/xw3dpsz136g3dxp6m1g312hh0000gq/T/ipykernel_30505/3473895911.py:37: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_onl

In [5]:
_yaml_file_contents=yaml.dump({'ECON-D':registerMap},allow_unicode=True, default_flow_style=False)

with open('ECOND_I2C_params_regmap.yaml','w') as _file:
    _file.write(_yaml_file_contents.replace("'",""))

In [6]:
d=pd.concat(newRegList)
d.drop(['Register Name','Register Width','Register Reset Value','Register Constraints','Field Description','Register Custom Type','Field Access','Field is Covered','Field is Reserved','Field is Volatile','Field Constraints','Register Description','addr_base','addr_offset'],axis=1,inplace=True)

d.rename(columns={'Register Address':'address',
 'Register Access':'access',
 'Field Name':'parameter',
 'Field Offset':'param_shift',
 'Field Width':'param_mask',
 'Field Reset Value':'default_value',
 'BlockName':'block'
},inplace=True)

In [7]:
blockSimple=d.block.str.replace('_ALL','').str.replace('RO_','').str.replace('WO_','').str.replace('RW_','').values

d['parameter_name']=blockSimple+"_"+d.parameter.values

blockInstance=np.array(['Global']*len(blockSimple))
for i in range(12):
    x=np.array([f'_{i:02n}' in x for x in blockSimple])
    blockInstance[x]=i
    blockSimple[x]=[v[:-3] for v in blockSimple[x]]
    
d['block']=blockSimple
d['block_id']=blockInstance

d['param_mask']=(2**d.param_mask)-1
d['max_value']=d.param_mask

In [8]:
isRSVD=np.array(['RSVD' in x for x in d.parameter])


In [9]:
cols='block,block_id,parameter,address,parameter_name,access,size_byte,default_value,param_mask,param_shift,max_value'.split(',')

d.loc[~isRSVD,cols].to_csv('ECOND_I2C_params_regmap.csv')

In [10]:
#map rename blocks (not yet done)
blockRenaming={'ALIGNER':'Aligner', 
 'CHAL':'ChAligner',
 'CHEPRXGRP':'ChEprxGrp',
 'CHERR':'ChErr',
 'CLOCKS_AND_RESETS':'ClocksAndResets',
 'ELINK_PROCESSORS':'ELinkProcessors',
 'EPRXGRP_TOP':'EprxGrpTop',
 'ERRTOP':'ErrTop',
 'ERX':'ERx',
 'ETX':'ETx',
 'FCTRL':'FCtrl',
 'FORMATTER_BUFFER':'FormatterBuffer',
 'MISC':'Misc',
 'MISC_TMR_ERR_CNT':'MiscTMRErrCnt',
 'PINGPONG_SRAM':'PingPongSRAM',
 'ROC_DAQ_CTRL':'RocDaqCtrl',
 'WATCHDOG':'Watchdog',
 'WATCHDOG_HAM':'WatchdogHam',
 'WATCHDOG_MISC':'WatchdogMisc',
 'WATCHDOG_SP':'WatchdogSP',
 'WATCHDOG_SUBP_CRC':'WatchdogSubPCRC',
 'WATCHDOG_SUBP_EBO':'WatchdogSubPEBO',
 'WATCHDOG_SUBP_HT':'WatchdogSubPHT',
 'ZESUPPRESS':'ZS',
 'ZESUPPRESS_COMMON':'ZSCommon',
 'ZESUPPRESS_M1':'ZSmOne'}